# Use Splash to Request ZocDoc Information

In [ ]:
from scrapy.http import TextResponse
import requests
from urllib.parse import urlencode


# you need start the docker
# url = 'https://www.zocdoc.com/doctor/ghulam-abbas-md-291989'
url = 'https://www.zocdoc.com/doctor/dane-salazar-md-246564'

# r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait':0})
proxyapi = '68cd99bcfa3c0879b87ea3052f61da4e'
r = requests.get('http://api.scraperapi.com/', params=urlencode({'api_key': proxyapi, 'url': url}))

response = TextResponse(r.url, body = r.text, encoding = 'utf-8')


In [ ]:
response.text

# Json Data

In [ ]:
import json
from pprint import pprint
xpath = './/script//text()'

json_data = response.xpath(xpath).extract()
len(json_data)

In [ ]:
json_string = [i for i in json_data if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]
json_info = [i for i in json_string.split(';\n') if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]

json_info = json_info.split('JSON.parse(')[1][:-1] # remove prefix
# x = x[:-1] # remove suffix
# json_info



x = json_info
x = json.loads(x)
x = json.loads(x)
pprint(x)

In [ ]:
doc_info = {}

## Root Info

In [ ]:
[# 'routing',
 # 'theme',
 # 'absystem',
 # '@location',
 # 'pageload',
 # '@@viewport',
 # 'prefetch',
 # 'user',
#  'signInForm',
 # '@@profile/in_network_insurance',
 # 'insurance-picker',
 # 'insurance',
 # '@@locationsCarousel',
 # 'NewExistingRadio',
 # '@@pictureGallery',
 'profile',
#  'seo',
 # 'patientPoweredSearch',
 # 'search',
 # '@@quickLinks',
 # 'timesgrid',
 # '@@videoVisitExplainerModal',
 # '@@profile/availability',
 # '@@mobileProfileBookingModal',
 # 'cookies'
]


# doc_info['timesgrid'] = x['timesgrid']
doc_info

## Profile Info

In [ ]:
col = 'profile'

profile = x[col]
[i for i in profile]
# x[col]

In [ ]:
cols = [# 'bookingSpotlightIsOn',
 # 'carrierId',
 # 'expandedLocations',
 # 'initParameters',
 # 'mapIsActive',
 # 'planId',
 # 'data',    # 'data',
 # 'dataVariables',
 'providerId',
 # 'deviceType',
#  'provider', # 'provider'
 # 'isNewPatient ',
 'isMentalHealthProvider',
 'doesProviderHaveInNetworkInsurances',
 # 'selectedLocationId',
#  'procedureId',
 'commonVirtualVisitReasons',
 # 'videoVisitExplainerModalType',
 'customDimensions',
 # 'defaultProcedureId',
#  'eventData',
 # 'host',
 # 'pageId',
 # 'pageCategory',
 # 'prefix',
 # 'seo',
#  'trustedInsuranceRating',
 'brandingVanityUrl']


for col in cols: doc_info[col] = profile[col]
    
pprint(doc_info)


### Profile Info - DataProvider

In [ ]:
profile_data_provider = profile['data']['provider']
# [i for i in profile_data_provider]

In [ ]:
cols = ['isOnlyInNetworkBookable','acceptsNewPatients','errorCode','id','monolithId','status',
 'profileUrl','isApproved','prenominal','postnominal','firstName',
 'lastName','directoryIds', 'approvedFullName', 'nameInSentence','newPatientAppointmentsAsIntroCalls',
 
 'genderIdentity', 'frontEndCirclePictureUrl', 'frontEndSquarePictureUrl', 'showPhotoWithCircleCss', 'smallPictures',
 'averageRating', 'averageBedsideRating','averageWaitTimeRating',
 'isPreview','personIdentifierSource', 'dataProvidedBy', 'dataProvidedByUrl',
 
 'mayChargeAdditionalCopay','onlySeesChildren', 'optOutOfRatings', 'previewSummary',
 'highlyRecommendPercentage', 'lowWaitTimedPercentage', 'hasNewPatientAvailability',
 'isTopRebookingProvider', 'hasVirtualLocations',
 'selfPaySettings', 'sellingPoints', 'approvedNonVirtualLocations', 'practices',
 'providerLocations', 'relevantSpecialty', 'mainSpecialtyId','specialties','approvedLocations',
 

 'statement', 'defaultProcedureId', 'hospitalAffiliations', 'certifications', 'education',
 'memberships', 'typedMemberships','awards','badges', 'languages','npi',
 'inNetworkInsurance','procedures','popularProcedures','isBookable','seo','reviewCount',
 'reviews', 'properties', 'trustedInsuranceRating', 'providerBadges','locations',
 'sortedLocations', 'numVirtualLocations',
 #  '__typename',
 # 'recentReviews',
]


# l = profile_data_provider['isOnlyInNetworkBookable']
# pprint(l)



for col in cols: doc_info[col] = profile_data_provider[col]
    
    
pprint(doc_info)
    

In [ ]:
print([i for i in doc_info])

In [ ]:
len(doc_info['reviews'])

In [ ]:
doc_info['complete_reviews'] = False if doc_info['reviewCount'] > len(doc_info['reviews']) else True

In [ ]:
doc_info['reviews']

# POST: URL (Failed)

In [ ]:

# import requests


# review_api_url = 'https://api.zocdoc.com/directory/v2/gql'
# providerID = 'pr_mw0z_x5HqE-YgjiprYcWax' # TODO
# limit = 100 # TODO

       
# forms = {
#     'operationName': 'getProviderReviews',
#     'query': "query getProviderReviews($providerId: String, $directoryId: String, $limit: Int, $sortReviewsBy: ReviewsSortBy, $sortReviewsDescending: Boolean) {\n  provider(id: $providerId, directoryId: $directoryId) {\n    id\n    ...providerReviews\n    __typename\n  }\n}\n\nfragment providerReviews on Provider {\n  reviews(limit: $limit, sortBy: $sortReviewsBy, sortDescending: $sortReviewsDescending) {\n    reviewId\n    bedsideRating\n    comment\n    date\n    dateRange\n    helpfulCount\n    isMailedInReview\n    isPartnerReview\n    isVirtualVisit\n    overallRating\n    patientName\n    waitTimeRating\n    __typename\n  }\n  __typename\n}\n", 
#     'variables':{
#         'directoryId': "-1",
#         "limit": str(limit),
#         "providerId": providerID,
#         "sortReviewsBy":"date",
#         "sortReviewsDescending": 'true',
#     }
# }


# r = requests.post('http://localhost:8050/render.html',
#               json={'url': review_api_url,
#                     'http_method': 'POST',
#                     'body': forms,
#                     'wait': 2})

# print(r.text)

# Selenium (when reviews > 50)

In [ ]:
# select your webdriver
from scrapy.http import HtmlResponse
from selenium import webdriver
import time

REVIEW_PER_PAGE = 50
webdriver_path = '_source/chromedriver_macm1'
reviewCount = doc_info['reviewCount']
    
    
    
def get_review(review_selector):
    d = {}
    d['comment'] = review_selector.xpath('.//*[@itemprop="reviewBody"]//span/text()').extract_first()
    d['date'] = review_selector.xpath('.//*[@itemprop="datePublished"]/text()').extract_first()
    d['patientName'] = review_selector.xpath('.//*[@itemprop="author"]/text()').extract_first()
    d['patient_type'] = review_selector.xpath('.//*[@data-test="review-source"]/text()').extract_first()
    d['overallRating'] = review_selector.xpath('.//*[@data-test="stars-svg-wrapper"]/svg/@data-rating').extract_first()
    return d


def get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path):
    options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument('--headless')

    driver = webdriver.Chrome(webdriver_path, options=options)
    driver.get(url)
    driver.title
    time.sleep(2)
    
    xpath = './/button[@data-test="reviews-read-more-button"]'
    
    for _ in range(int(reviewCount / REVIEW_PER_PAGE )):
        more_result_button = driver.find_element_by_xpath(xpath)
        print(more_result_button)
        driver.execute_script("arguments[0].click();", more_result_button)
        time.sleep(1)
        
    response = HtmlResponse(driver.current_url, body=driver.page_source, encoding='utf-8')
    reviews = response.xpath('//div[@itemprop="reviews"]')
    reviews = [get_review(i) for i in reviews]
    # len(reviews)
    driver.quit()
    return reviews



reviews = get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path)

print('\nReviews: reported {} vs collected {}'.format(reviewCount, len(reviews)))



# All In One

In [ ]:
import os
import requests
import pandas as pd
import time
import requests
from scrapy.http import TextResponse
import json
import html
import argparse
import random
from datetime import datetime
from scrapy.http import HtmlResponse
from selenium import webdriver
import time
from datetime import datetime



headers = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
}
        
def get_review(review_selector):
    d = {}
    d['comment'] = review_selector.xpath('.//*[@itemprop="reviewBody"]//span/text()').extract_first()
    d['date'] = review_selector.xpath('.//*[@itemprop="datePublished"]/text()').extract_first()
    d['patientName'] = review_selector.xpath('.//*[@itemprop="author"]/text()').extract_first()
    d['patient_type'] = review_selector.xpath('.//*[@data-test="review-source"]/text()').extract_first()
    d['overallRating'] = review_selector.xpath('.//*[@data-test="stars-svg-wrapper"]/svg/@data-rating').extract_first()
    return d

def get_complete_reviews_from_selenium(url, reviewCount, REVIEW_PER_PAGE, webdriver_path):
    options = webdriver.ChromeOptions()
    # options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument('--headless')

    driver = webdriver.Chrome(webdriver_path, options=options)
    driver.get(url)
    driver.title
    time.sleep(2)
    
    xpath = './/button[@data-test="reviews-read-more-button"]'
    
    for _ in range(int(reviewCount / REVIEW_PER_PAGE )):
        more_result_button = driver.find_element_by_xpath(xpath)
        print(more_result_button)
        driver.execute_script("arguments[0].click();", more_result_button)
        time.sleep(1)
        
    response = HtmlResponse(driver.current_url, body=driver.page_source, encoding='utf-8')
    reviews = response.xpath('//div[@itemprop="reviews"]')
    reviews = [get_review(i) for i in reviews]
    # len(reviews)
    driver.quit()
    return reviews


def get_physician_info_from_zocdoc_url(ph_url, REVIEW_PER_PAGE, webdriver_path, use_webdriver):
    doc_info = {}
    
    r = requests.get('http://localhost:8050/render.html', params={'url': ph_url, 'wait':0})
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
    
    xpath = './/script//text()'

    json_data = response.xpath(xpath).extract()
    # len(json_data)

    json_string = [i for i in json_data if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]
    json_info = [i for i in json_string.split(';\n') if 'window.__REDUX_STATE__ = JSON.parse(' in i][0]

    json_info = json_info.split('JSON.parse(')[1][:-1] # remove prefix
    # x = x[:-1] # remove suffix
    # json_info

    x = json_info
    x = json.loads(x)
    x = json.loads(x) # yes, double json.load operations


    # (1) basic profile
    col = 'profile'; profile = x[col]

    cols = [ 'providerId', 'isMentalHealthProvider',
        'doesProviderHaveInNetworkInsurances', 
        'commonVirtualVisitReasons', 'customDimensions', 'brandingVanityUrl']

    for col in cols: doc_info[col] = profile[col]

    # (2) provider
    profile_data_provider = profile['data']['provider']

    cols = ['isOnlyInNetworkBookable','acceptsNewPatients','errorCode','id','monolithId','status',
        'profileUrl','isApproved','prenominal','postnominal','firstName',
        'lastName','directoryIds', 'approvedFullName', 'nameInSentence','newPatientAppointmentsAsIntroCalls',
        'genderIdentity', 'frontEndCirclePictureUrl', 'frontEndSquarePictureUrl', 'showPhotoWithCircleCss', 'smallPictures',
        'averageRating', 'averageBedsideRating','averageWaitTimeRating',
        'isPreview','personIdentifierSource', 'dataProvidedBy', 'dataProvidedByUrl',
        'mayChargeAdditionalCopay','onlySeesChildren', 'optOutOfRatings', 'previewSummary',
        'highlyRecommendPercentage', 'lowWaitTimedPercentage', 'hasNewPatientAvailability',
        'isTopRebookingProvider', 'hasVirtualLocations',
        'selfPaySettings', 'sellingPoints', 'approvedNonVirtualLocations', 'practices',
        'providerLocations', 'relevantSpecialty', 'mainSpecialtyId','specialties','approvedLocations',
        'statement', 'defaultProcedureId', 'hospitalAffiliations', 'certifications', 'education',
        'memberships', 'typedMemberships','awards','badges', 'languages','npi',
        'inNetworkInsurance','procedures','popularProcedures','isBookable','seo','reviewCount',
        'reviews', 'properties', 'trustedInsuranceRating', 'providerBadges','locations',
        'sortedLocations', 'numVirtualLocations',]

    for col in cols: doc_info[col] = profile_data_provider[col]

    doc_info['complete_reviews'] = False if doc_info['reviewCount'] > len(doc_info['reviews']) else True


    reviewCount = doc_info['reviewCount']

    if doc_info['complete_reviews'] == False and use_webdriver == True:
        reviews = get_complete_reviews_from_selenium(ph_url, reviewCount, 
                                                    REVIEW_PER_PAGE, webdriver_path)

        print('\nReviews: reported {} vs collected {}'.format(reviewCount, len(reviews)))
        if len(reviews) == reviewCount: doc_info['complete_reviews'] = True 
        
        doc_info['reviews'] = reviews
    
    return doc_info


In [ ]:
ph_url = url
REVIEW_PER_PAGE = 50
webdriver_path = '_source/chromedriver_macm1'

doc_info = get_physician_info_from_zocdoc_url(ph_url, REVIEW_PER_PAGE, webdriver_path)

In [ ]:
len(doc_info['reviews'])

In [ ]:
doc_info['reviews']